# Devoir 4, Question 1 : Réseau de neurones à convolution

# Homework 4, Question 1: Convolutional neural network

## Code préambule

## Preamble code

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import gzip
import pandas
import time
import numpy

from IPython import display

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import torchvision.transforms as T

import matplotlib
matplotlib.rcParams['figure.figsize'] = (9.0, 7.0)
from matplotlib import pyplot

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def create_balanced_sampler(dataset):
    def make_weights_for_balanced_classes(images, n_classes):                        
        count = [0] * n_classes                                                      
        for item in images:                                                         
            count[item[1]] += 1                                                     
        weight_per_class = [0.] * n_classes                                      
        N = float(sum(count))                                                   
        for i in range(n_classes):                                                   
            weight_per_class[i] = N/float(count[i])                                 
        weight = [0] * len(images)                                              
        for idx, val in enumerate(images):                                          
            weight[idx] = weight_per_class[val[1]]                                  
        return weight

    n_classes = numpy.unique(dataset.targets)
    weights = make_weights_for_balanced_classes(dataset.data, len(n_classes))                                                         
    weights = torch.DoubleTensor(weights)                 
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) 
    return sampler

def compute_accuracy(model, dataloader, device='cpu'):
    training_before = model.training
    model.eval()
    all_predictions = []
    all_targets = []
    
    for i_batch, batch in enumerate(dataloader):
        images, targets = batch
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            predictions = model(images)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    if all_predictions[0].shape[-1] > 1:
        predictions_numpy = numpy.concatenate(all_predictions, axis=0)
        predictions_numpy = predictions_numpy.argmax(axis=1)
        targets_numpy = numpy.concatenate(all_targets, axis=0)
    else:
        predictions_numpy = numpy.concatenate(all_predictions).squeeze(-1)
        targets_numpy = numpy.concatenate(all_targets)
        predictions_numpy[predictions_numpy >= 0.5] = 1.0
        predictions_numpy[predictions_numpy < 0.5] = 0.0

    if training_before:
        model.train()

    return (predictions_numpy == targets_numpy).mean()


Pour cette question, vous devez faire l'entraînement d'un réseau de neurones sur le jeu de données [*Volcanoes on Venus*](https://www.kaggle.com/fmena14/volcanoesvenus/). Il s'agit d'un problème de classification pour lequel nous vous fournissons une version abrégée du jeu de données d'[entraînement](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/volcanoes_train.pt.gz) et de [test](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/volcanoes_test.pt.gz) (attention: 120 Mo, les fichiers sont directement disponibles sur PAX).

Pour vous mettre en contexte, voici une synthèse des trois étapes requises pour l'entraînement de votre modèle dans un contexte de classification:

1. **Gérer les données:** La première étape est la gestion des données. Elle se fait en deux temps. En premier, il faut définir une classe (dans le sens *programmation orientée objet* du terme) qui s'occupe des données. Elle permet de contrôler le chargement et l'application des transformations. Chaque fois qu'une donnée (ici une image) est demandée par le système, cette classe est appelée. Cette classe permet de faire une copie des images du disque dur vers la mémoire vive de votre machine. Ce transfert est un moment propice pour appliquer les transformations, tout en réduisant le temps de déplacement des données vers le GPU. Dans *PyTorch*, ceci est effectué par une classe dénommée `Dataset`. Ensuite, il faut définir une classe, nommée `DataLoader` dans *PyTorch*, qui contrôle la façon dont les données sont sélectionnées dans le jeu de données, car on doit pouvoir décider si on veut les piger aléatoirement ou dans un ordre particulier. Elle permet alors de définir la méthode d'échantillonnage et la taille des lots (*batch*) que l'on souhaite obtenir. Notez également que *PyTorch* fonctionne principalement avec des [tenseurs](https://fr.wikipedia.org/wiki/Tenseur) -- généralisation à plusieurs dimensions des matrices.

2. **Développer le modèle:** La seconde étape est de définir le modèle de réseau de neurones que l'on souhaite utiliser. Ce réseau peut être construit et personnalisé dans une classe *PyTorch*, que l'on nomme `VolcanoesNet` pour la question courante. Elle permet de définir et d'initialiser les couches du réseau dans la fonction `init`. La fonction `forward` permet de contrôler dans quel ordre se fera l'inférence sur les couches définies dans `init`. Elle permet aussi de varier la forme du tenseur entre les couches, au besoin.

3. **Entraîner le modèle:** La dernière étape est d'entraîner le modèle. Pour ce faire, vous devez développer les boucles d'entraînement. On entraîne un modèle itératif, où une époque représente une boucle complète sur toutes les données d'entraînement et une *batch* représente un lot de données utilisées pour une inférence, échantillonnées par le `DataLoader`. Pour chaque *batch*, les opérations d'entraînement d'un réseau (remise à zéro des gradients, inférence, calcul de la perte, rétropropagation) doivent être appliquées. La séparation du jeu de données en *batch* permet de ne pas dépasser la capacité mémoire des GPUs, comme on traite chacune d'entre elles indépendamment. Pour chaque *batch*, les données sont transférées de la mémoire vive du CPU vers le GPU (et inversement à la fin de la *batch*) pour appliquer les opérations d'entraînement.

For this question, you have to train a neural network on the [*Volcanoes on Venus*](https://www.kaggle.com/fmena14/volcanoesvenus/) dataset. This is a classification problem for which we provide you with an abridged version of the [training](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/volcanoes_train.pt.gz) and [testing](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/volcanoes_test.pt.gz) datasets (warning: 120 MB, the files are directly available on PAX).

To put you in context, here is a summary of the three steps required to train your model in a classification context:

1. **Data management:** The first step is data management. It is done in two phases. First, you need to define a class (in the *object-oriented programming* meaning of the term) that deals with the data. It allows to control the loading and the application of the transformations. Each time a data element (here an image) is requested by the system, this class is called. This class allows you to make a copy of the images from the hard disk to the RAM of your machine. This transfer is a good time to apply the transformations, while reducing the time taken to move the data to the GPU. In *PyTorch*, this is done by a class called `Dataset`. Next, we need to define a class, named `DataLoader` in *PyTorch*, which controls how the data is selected from the dataset, as we need to be able to decide whether to pick them randomly or in a particular order. It then allows you to define the sampling method and the size of the batches that you want to obtain. Note also that *PyTorch* works mainly with [tensors](https://en.wikipedia.org/wiki/Tensor) -- a generalization to several dimensions of matrices.

2. **Model development** The second step is to define the neural network model that you want to use. This network can be built and customized in a *PyTorch* class, which we name `VolcanoesNet` for the current question. It allows to define and initialize the layers of the network with the `init` function. The function `forward` allows to control the order of the inference on the layers defined in `init`. It also allows to vary the shape of the tensor between layers, as needed.

3. **Train the model:** The last step is to train the model. To do this, you need to develop the training loops. We train an iterative model, where an epoch represents a complete loop over all the training data and a *batch* represents a bunch of data used for an inference, sampled by the `DataLoader`. For each *batch*, the training operations of a network (gradient reset, inference, loss calculation, backpropagation) must be applied. The separation of the dataset into batches allows to not exceed the memory capacity of the GPUs, as each of them is processed independently. For each batch, the data is transferred from the CPU to the GPU (and vice versa at the end of the *batch*) to apply the training operations.

## Q1A
Pour commencer, vous vous familiarisez avec les données Volcanoes afin de pouvoir les manipuler dans l'entraînement. Pour ce faire, définissez la classe `VolcanoesDataset`, qui hérite de la classe abstraite `torch.utils.data.Dataset`, et surchargez les méthodes `__getitem__` et `__len__`, comme mentionné dans la [documentation](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset). Ceci doit résulter en un jeu de données utilisable par *PyTorch*.

De plus, vous devez tester votre classe `VolcanoesDataset` en affichant quatre images choisies **aléatoirement**, dans une figure unique. Indiquez la classe correspondante dans le titre de chacune des sous-figures. Également, vous devez représenter la distribution des données par classe du jeu d'entraînement dans un histogramme.

## Q1A 
To begin, you familiarize yourself with the Volcanoes dataset so that you can manipulate it for the training. To do this, define the class `VolcanoesDataset`, which inherits from the abstract class `torch.utils.data.Dataset`, and overload the methods `__getitem__` and `__len__`, as mentioned in the [documentation](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset). This should result in a dataset usable by *PyTorch*.

In addition, you should test your `VolcanoesDataset` class by displaying four **randomly** chosen images in a single figure. Indicate the corresponding class in the title of each of the subfigures. Also, you need to represent the class distribution of the training set data in a histogram.

### Patron de code réponse à l'exercice Q1A

### Q1A answer code template

In [ ]:
class VolcanoesDataset(Dataset):
    """
    Cette classe sert à définir le dataset Volcanoes pour PyTorch
    proposé de Francisco Mena sur kaggle : https://bit.ly/2DasPF1

    Args:
        path (str): le chemin du fichier .pt du dataset

    This class is used to define the Volcanoes dataset for PyTorch
    proposed by Francisco Mena on kaggle : https://bit.ly/2DasPF1

    Args:
        path (str): path to dataset .pt file 
    """

    def __init__(self, path):
        super().__init__()
        # garde les paramètres en mémoire / store parameters in memory
        self.path = path
        # charger les données / load data
        with gzip.open(path, 'rb') as f:
            self.data = torch.load(f)
        # Pour faciliter la lecture des valeurs cibles / ease reading the targets
        self.targets = numpy.array(list(zip(*self.data))[1])

    def __getitem__(self, index):
        # *** TODO ***
        # Fourni l'instance à un certain indice du jeu de données
        # Provide an instance of the dataset according to the index
        pass # Retirer le pass / remove the pass
        # ******

    
    def __len__(self):
        # *** TODO ***
        # Fournis la taille du jeu de données
        # Provide the lenght of the dataset
        pass # Retirer le pass / remove the pass
        # ******

        
# Creation du dataset / Creating the dataset
train_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_train.pt.gz')

fig, subfigs = pyplot.subplots(2, 2, tight_layout=True)
for subfig in subfigs.reshape(-1):
    # *** TODO ***
    # Affichage de quatre images aléatoires
    # Displaying four random images
    pass # Retirer le pass / remove the pass
    # ******

fig, subfig = pyplot.subplots()

# *** TODO ***
# Tracer histogramme de la distribution des données par classe de train_set
# Plot class distribution histogram for train_set
# ******

### Entrez votre solution à Q1A dans la cellule ci-dessous

### Enter your answer to Q1A in the cell below.

<div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
                <h3>Votre soumission a été enregistrée!</h3>
                <ul>
                    <li>notez qu'il n'y a <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
                    <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
                    <li>elle sera cependant ajustée par le professeur dès que la correction manuelle sera complétée&puncsp;;</li>
                    <li>vous pouvez soumettre autant de fois que nécessaire jusqu'à la date d'échéance&puncsp;;</li>
                    <li>mais évitez de soumettre inutilement.</li>
                </ul>
            </div>

In [ ]:
class VolcanoesDataset(Dataset):
    """
    Cette classe sert à définir le dataset Volcanoes pour PyTorch
    proposé de Francisco Mena sur kaggle : https://bit.ly/2DasPF1

    Args:
        path (str): le chemin du fichier .pt du dataset

    This class is used to define the Volcanoes dataset for PyTorch
    proposed by Francisco Mena on kaggle : https://bit.ly/2DasPF1

    Args:
        path (str): path to dataset .pt file 
    """

    def __init__(self, path):
        super().__init__()
        # garde les paramètres en mémoire / store parameters in memory
        self.path = path
        # charger les données / load data
        with gzip.open(path, 'rb') as f:
            self.data = torch.load(f)
        # Pour faciliter la lecture des valeurs cibles / ease reading the targets
        self.targets = numpy.array(list(zip(*self.data))[1])

    def __getitem__(self, index):
        # *** TODO ***
        # Fourni l'instance à un certain indice du jeu de données
        # Provide an instance of the dataset according to the index
#         if isinstance(index, slice):
#             indices = range(*index.indices(len(self.data)))
#             return [self.data[n][0] for n in indices]
        return self.data[index][0], self.targets[index]
        # ******

    
    def __len__(self):
        # *** TODO ***
        # Fournis la taille du jeu de données
        # Provide the lenght of the dataset
        return len(self.data)
        # ******

        
# Creation du dataset / Creating the dataset
train_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_train.pt.gz')

fig, subfigs = pyplot.subplots(2, 2, tight_layout=True)
for subfig in subfigs.reshape(-1):
    # *** TODO ***
    # Affichage de quatre images aléatoires
    # Displaying four random images
    subfig.axis("off")
    subfig.imshow(train_set[numpy.random.randint(7000, size=1)[0]][0].permute(1, 2, 0), cmap="gray_r")
    # ******

fig, subfig = pyplot.subplots()

# *** TODO ***
# Tracer histogramme de la distribution des données par classe de train_set
# Plot class distribution histogram for train_set
# ******
y = train_set.targets
labels, counts = numpy.unique(y, return_counts=True)
subfig.hist(labels, bins=[-0.5, 0.5, 1.5], weights=counts)
subfig.set_xticks(labels)

## Q1B
Vous devez maintenant créer le réseau de neurones et définir les méthodes ainsi que les attributs nécessaires pour qu'il puisse être entraîné.
- Commencez par initialiser les couches de votre réseau dans la méthode `__init__` de `VolcanoesNet`, en utilisant les couches de convolution (`Conv2D`), de normalisation (`BatchNorm2D`) et linéaire (`Linear`), selon l'architecture suivante.
![Architecture de VolcanoesNet](https://pax.ulaval.ca/static/GIF-4101-7005/images/d4q1_volcanoes_net.png)
- Pour les convolutions, vous devez respecter le nombre de filtres (*filters*) et la taille des noyaux (*kernels*) de convolution. Vous devez aussi, et ce pour toutes les convolutions, spécifier un pas (*stride*) de 2. Aussi, vous devez retirer le biais de la convolution si cette dernière est suivie d'une couche de normalisation, car elle contient déjà un paramètre pour le biais.
- Écrivez les lignes de code manquantes pour définir l'ordre d'inférence des couches dans le réseau dans la méthode `forward` de `VolcanoesNet`. Les modules `average_pooling`, `linear` et `sigmoid` sont déjà implémentées dans la librairie [PyTorch](https://pytorch.org/docs/stable/nn.html).

## Q1B
You now need to create the neural network and define the methods and attributes needed to train it.
- Start by initializing the layers of your network in the `__init__` method of `VolcanoesNet`, using the convolution (`Conv2D`), normalization (`BatchNorm2D`) and linear (`Linear`) layers, according to the following architecture.
![VolcanoesNet Architecture](https://pax.ulaval.ca/static/GIF-4101-7005/images/d4q1_volcanoes_net.png)
- For convolutions, you must respect the number of filters and the size of the convolution kernels. You must also, for all convolutions, specify a stride of 2. Also, you must remove the bias from the convolution if it is followed by a normalization layer, since it already contains a parameter for the bias.
- Write the missing lines of code to define the order of inference of the layers in the network in the `forward` method of `VolcanoesNet`. The modules `average_pooling`, `linear` and `sigmoid` are already implemented in the [PyTorch] library (https://pytorch.org/docs/stable/nn.html).

### Patron de code réponse à l'exercice Q1B

### Q1B answer code template

In [ ]:
class VolcanoesNet(nn.Module):
    """
    Cette classe définit un réseau pleinement convolutionnel simple
    permettant de classifier des images satellite de Venus.
    This class defines a simple fully convolutional network
    to classify satellite images from Venus.
    """

    def __init__(self):
        super().__init__()
        # *** TODO ***
        # Initialiser ici les modules contenant des paramètres à optimiser.
        # Ces modules seront utilisés dans la méthode 'forward'
        pass # Retirer le pass / Remove the pass
        # ******


    def forward(self, x):
        # *** TODO ***
        # Effectuer l'inférence du réseau. L'ordre d'exécution est importante.
        # Perform network inference. The order of execution is important.
        return False  # Retourner la bonne valeur / return the right value
        # ******

### Entrez votre solution à Q1B dans la cellule ci-dessous

### Enter your answer to Q1B in the cell below.

In [ ]:
class VolcanoesNet(nn.Module):
    """
    Cette classe définit un réseau pleinement convolutionnel simple
    permettant de classifier des images satellite de Venus.
    This class defines a simple fully convolutional network
    to classify satellite images from Venus.
    """

    def __init__(self):
        super().__init__()
        # *** TODO ***
        # Initialiser ici les modules contenant des paramètres à optimiser.
        # Ces modules seront utilisés dans la méthode 'forward'
        super().__init__()
        self.C1 = nn.Conv2d(1, 32, kernel_size=5, stride=2, bias=False)
        self.B2 = nn.BatchNorm2d(32)
        self.C3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, bias=False)
        self.B4 = nn.BatchNorm2d(64)
        self.C5 = nn.Conv2d(64, 64, kernel_size=3, stride=2, bias=False)
        self.B6 = nn.BatchNorm2d(64)
        self.C7 = nn.Conv2d(64, 64, kernel_size=3, stride=2, bias=False)
        self.B8 = nn.BatchNorm2d(64)
        self.C9 = nn.Conv2d(64, 64, kernel_size=3, stride=2, bias=False)
        self.B10 = nn.BatchNorm2d(64)
        self.A11 = nn.AvgPool2d(2)
        self.output = nn.Linear(64, 1)
        # ******


    def forward(self, x):
        # *** TODO ***
        # Effectuer l'inférence du réseau. L'ordre d'exécution est importante.
        # Perform network inference. The order of execution is important.
        x = F.relu(self.B2(self.C1(x)))
        x = F.relu(self.B4(self.C3(x)))
        x = F.relu(self.B6(self.C5(x)))
        x = F.relu(self.B8(self.C7(x)))
        x = self.B10(self.C9(x))
        x = self.A11(x)
        x = x.view(-1, 64)
        return torch.sigmoid(self.output(x))  # Retourner la bonne valeur / return the right value
        # ******

## Q1C
Il faut maintenant développer les outils nécessaires pour effectuer l'entraînement du réseau de neurones, selon le code que vous avez développé aux sous-questions précédentes. L'entraînement est défini par une boucle qui itère sur l'ensemble des données d'entraînement, chaque itération correspondant à une époque. Pour chaque époque, il faut itérer sur tous les lots (*batch*) qu'elle contient.

Pour cette question, vous devez:
 - Écrire le code manquant pour la préparation de l'entraînement.
 - Écrire le code manquant à l'intérieur de la boucle d'entraînement.
 - Écrire le code manquant à l'intérieur de la fonction de calcul de l'erreur et des matrices de confusion.

La matrice de confusion est particulièrement utile pour visualiser les performances de votre réseau. On assigne la donnée à la première classe 0 lorsque la probabilité en sortie est plus petite que 0,5, sinon la données est assignée à la deuxième classe. 

Également, discutez brièvement les performances du réseau selon la matrice de confusion obtenue.

## Q1C
We now need to develop the tools necessary to perform the training of the neural network, according to the code you developed in the previous sub-questions. The training is defined by a loop that iterates over the whole training data, each iteration corresponding to an epoch. For each epoch, you must iterate over all the batches it contains.

For this question, you must:
 - Write the missing code for the training setup.
 - Write the missing code inside the training loop.
 - Write the missing code inside the error calculation function and confusion matrices.

The confusion matrix is particularly useful for visualizing the performance of your network. The data is assigned to the first class 0 when the output probability is smaller than 0.5, otherwise the data is assigned to the second class. 

Also, briefly discuss the performance of the network according to the obtained confusion matrix.

In [ ]:
# Initialisation des paramètres d'entraînement
# Paramètres recommandés:
# - Nombre d'epochs (nb_epoch = 10)
# - Taux d'apprentissage (learning_rate = 0.01)
# - Momentum (momentum = 0.9)
# - Taille du lot (batch_size = 32)
#
# Initialization of training parameters
# Recommended parameters:
# - Number of epochs (nb_epoch = 10)
# - Learning rate (learning_rate = 0.01)
# - Momentum (momentum = 0.9)
# - Batch size (batch_size = 32)
nb_epoch = 10
learning_rate = 0.01
momentum = 0.9
batch_size = 32

# Chargement des données d'entraînement et de test
# Loading training and testing set
train_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_train.pt.gz')
test_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_test.pt.gz')

# Création du sampler avec les classes balancées
# Create the sampler with balanced classes
balanced_train_sampler = create_balanced_sampler(train_set)
balanced_test_sampler = create_balanced_sampler(test_set)

# Création du dataloader d'entraînement
# Create training dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, sampler=balanced_train_sampler)
test_loader = DataLoader(test_set, batch_size=batch_size, sampler=balanced_test_sampler)

def compute_confusion_matrix(model, dataloader, device):
    
    # *** TODO ***
    # Mettre le model en mode évaluation
    # Calculer toutes les prédictions sur le dataloader
    # Put the model in evaluation mode
    # Compute all predictions on the dataloader  
    all_predictions = []
    all_targets = []
    for i_batch, batch in enumerate(dataloader):
        images, targets = batch
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            predictions = model(images)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    predictions_numpy = numpy.concatenate(all_predictions)
    targets_numpy = numpy.concatenate(all_targets)
    # ******

    # *** TODO ***
    # Assigner la classe 0 ou 1 aux prédictions
    # Calculer la matrice de confusion. Attention de bien avoir
    # une matrice 2 par 2 en sortie
    #
    # Assign class 0 or 1 to the predictions
    # Compute the confusion matrix. Be careful to have
    # a 2 by 2 matrix as output.
    # ******

    return matrix  # Retourner matrice de confusion / return confusion matrix


# *** TODO ***
# Instancier votre réseau VolcanoesNet dans une variable nommée "model"
# Instantiate your VolcanoesNet network in a variable named "model"
# ******

# Transférer le réseau sur GPU ou CPU en fonction de la variable "DEVICE"
# Transfer the network to GPU or CPU depending on the "DEVICE" variable
model.to(DEVICE)

# *** TODO ***
# Instancier une fonction d'erreur BinaryCrossEntropy
# et la mettre dans une variable nommée criterion
# Instantiate an error function BinaryCrossEntropy
# and put it in a variable named criterion

# Instancier l'algorithme d'optimisation SGD
# Ne pas oublier de lui donner les hyperparamètres
# d'entraînement : learning rate et momentum!
# Instantiate the SGD optimization algorithm
# Don't forget to give it the training hyperparameters:
# learning rate and momentum!

# Mettre le réseau en mode entraînement
# Set the network in training mode
# ******

# Boucle d'entraînement / Training loop
for i_epoch in range(nb_epoch):

    start_time, train_losses = time.time(), []
    for i_batch, batch in enumerate(train_loader):
        images, targets = batch
        targets = targets.type(torch.FloatTensor).unsqueeze(-1)

        images = images.to(DEVICE)
        targets = targets.to(DEVICE)
        
        # *** TODO ***
        # Mettre les gradients à zéro
        # Set gradients to zero

        # Calculer:
        # 1. l'inférence dans une variable "predictions"
        # 2. l'erreur dans une variable "loss"
        # Compute:
        # 1. the inference in a "predictions" variable
        # 2. the error in a "loss" variable

        # Rétropropager l'erreur et effectuer
        # une étape d'optimisation
        # Backpropagate the error and perform
        # an optimization step
        # ******
        
        # Accumulation du loss de la batch
        # Accumulating batch loss
        train_losses.append(loss.item())

    print(' [-] epoch {:4}/{:}, train loss {:.6f} in {:.2f}s'.format(
        i_epoch+1, nb_epoch, numpy.mean(train_losses), time.time()-start_time))

# Affichage du score en test / Display test score
test_acc = compute_accuracy(model, test_loader, DEVICE)
print(' [-] test acc. {:.6f}%'.format(test_acc * 100))

# Affichage de la matrice de confusion / Display confusion matrix
matrix = compute_confusion_matrix(model, test_loader, DEVICE)
print(matrix)

# Libère la cache sur le GPU *important sur un cluster de GPU*
# Free GPU cache *important on a GPU cluster*
torch.cuda.empty_cache()

# *** TODO ***
# Entrez vos commentaires de la discussion ici.
# Enter your discussion comments here
discussion = "Entrez vos commentaires de la discussion ici."
# ******

frame = {"Comments":[discussion]}
df = pandas.DataFrame(frame)
display.display(df)

### Entrez votre solution à Q1C dans la cellule ci-dessous

### Enter your answer to Q1C in the cell below.

In [ ]:
# Initialisation des paramètres d'entraînement
# Paramètres recommandés:
# - Nombre d'epochs (nb_epoch = 10)
# - Taux d'apprentissage (learning_rate = 0.01)
# - Momentum (momentum = 0.9)
# - Taille du lot (batch_size = 32)
#
# Initialization of training parameters
# Recommended parameters:
# - Number of epochs (nb_epoch = 10)
# - Learning rate (learning_rate = 0.01)
# - Momentum (momentum = 0.9)
# - Batch size (batch_size = 32)
nb_epoch = 10
learning_rate = 0.01
momentum = 0.9
batch_size = 32

# Chargement des données d'entraînement et de test
# Loading training and testing set
train_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_train.pt.gz')
test_set = VolcanoesDataset('/pax/shared/GIF-4101-7005/volcanoes_test.pt.gz')

# Création du sampler avec les classes balancées
# Create the sampler with balanced classes
balanced_train_sampler = create_balanced_sampler(train_set)
balanced_test_sampler = create_balanced_sampler(test_set)

# Création du dataloader d'entraînement
# Create training dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, sampler=balanced_train_sampler)
test_loader = DataLoader(test_set, batch_size=batch_size, sampler=balanced_test_sampler)

def compute_confusion_matrix(model, dataloader, device):
    
    # *** TODO ***
    # Mettre le model en mode évaluation
    # Calculer toutes les prédictions sur le dataloader
    # Put the model in evaluation mode
    # Compute all predictions on the dataloader
    all_predictions = []
    all_targets = []
    for i_batch, batch in enumerate(dataloader):
        images, targets = batch
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            predictions = model(images)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    predictions_numpy = numpy.concatenate(all_predictions)
    targets_numpy = numpy.concatenate(all_targets)
    # ******

    # *** TODO ***
    # Assigner la classe 0 ou 1 aux prédictions
    # Calculer la matrice de confusion. Attention de bien avoir
    # une matrice 2 par 2 en sortie
    #
    # Assign class 0 or 1 to the predictions
    # Compute the confusion matrix. Be careful to have
    # a 2 by 2 matrix as output.
    # ******
    if all_predictions[0].shape[-1] > 1:
        predictions_numpy = numpy.concatenate(all_predictions, axis=0)
        predictions_numpy = predictions_numpy.argmax(axis=1)
        targets_numpy = numpy.concatenate(all_targets, axis=0)
    else:
        predictions_numpy = numpy.concatenate(all_predictions).squeeze(-1)
        targets_numpy = numpy.concatenate(all_targets)
        predictions_numpy[predictions_numpy >= 0.5] = 1.0
        predictions_numpy[predictions_numpy < 0.5] = 0.0
        
    matrix = numpy.array([
     numpy.sum(numpy.logical_and(predictions_numpy == 0, targets_numpy == 0)), 
     numpy.sum(numpy.logical_and(predictions_numpy == 0, targets_numpy == 1)),
     numpy.sum(numpy.logical_and(predictions_numpy == 1, targets_numpy == 0)),
     numpy.sum(numpy.logical_and(predictions_numpy == 1, targets_numpy == 1)),
    ]).reshape((2, 2))

    return matrix  # Retourner matrice de confusion / return confusion matrix


# *** TODO ***
# Instancier votre réseau VolcanoesNet dans une variable nommée "model"
# Instantiate your VolcanoesNet network in a variable named "model"
# ******
model = VolcanoesNet()

# Transférer le réseau sur GPU ou CPU en fonction de la variable "DEVICE"
# Transfer the network to GPU or CPU depending on the "DEVICE" variable
model.to(DEVICE)

# *** TODO ***
# Instancier une fonction d'erreur BinaryCrossEntropy
# et la mettre dans une variable nommée criterion
# Instantiate an error function BinaryCrossEntropy
# and put it in a variable named criterion
criterion = nn.BCELoss()

# Instancier l'algorithme d'optimisation SGD
# Ne pas oublier de lui donner les hyperparamètres
# d'entraînement : learning rate et momentum!
# Instantiate the SGD optimization algorithm
# Don't forget to give it the training hyperparameters:
# learning rate and momentum!
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# Mettre le réseau en mode entraînement
# Set the network in training mode
# ******
model.train()

# Boucle d'entraînement / Training loop
for i_epoch in range(nb_epoch):

    start_time, train_losses = time.time(), []
    for i_batch, batch in enumerate(train_loader):
        images, targets = batch
        targets = targets.type(torch.FloatTensor).unsqueeze(-1)

        images = images.to(DEVICE)
        targets = targets.to(DEVICE)
        
        # *** TODO ***
        # Mettre les gradients à zéro
        # Set gradients to zero
        optimizer.zero_grad()

        # Calculer:
        # 1. l'inférence dans une variable "predictions"
        # 2. l'erreur dans une variable "loss"
        # Compute:
        # 1. the inference in a "predictions" variable
        # 2. the error in a "loss" variable
        output = model(images)
        loss = criterion(output, targets)

        # Rétropropager l'erreur et effectuer
        # une étape d'optimisation
        # Backpropagate the error and perform
        # an optimization step
        # ******
        loss.backward()
        optimizer.step()
        
        # Accumulation du loss de la batch
        # Accumulating batch loss
        train_losses.append(loss.item())

    print(' [-] epoch {:4}/{:}, train loss {:.6f} in {:.2f}s'.format(
        i_epoch+1, nb_epoch, numpy.mean(train_losses), time.time()-start_time))

# Affichage du score en test / Display test score
test_acc = compute_accuracy(model, test_loader, DEVICE)
print(' [-] test acc. {:.6f}%'.format(test_acc * 100))

# Affichage de la matrice de confusion / Display confusion matrix
matrix = compute_confusion_matrix(model, test_loader, DEVICE)
print(matrix)

# Libère la cache sur le GPU *important sur un cluster de GPU*
# Free GPU cache *important on a GPU cluster*
torch.cuda.empty_cache()

# *** TODO ***
# Entrez vos commentaires de la discussion ici.
# Enter your discussion comments here
discussion = "Je constate que le résultat présenté par la matrice de confusion\
              est légèrement supérieur au résultat affiché par test_acc"
# ******

frame = {"Comments":[discussion]}
df = pandas.DataFrame(frame)
display.display(df)